In [1]:
!pip install rdflib
!pip install SPARQLWrapper

In [2]:
!pip install tqdm

ERROR: Operation cancelled by user


## Initialization code

In [2]:
from bs4 import BeautifulSoup
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
import requests
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS

g = rdflib.Graph()
g.parse("movie.rdf", format="xml")
MOVIE = Namespace("http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#")
g.bind("movie", MOVIE)
bearer_token = "xxx"


In [4]:
MOVIE_NS = Namespace("http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#")

In [5]:
def tmdb_api_request(endpoint, params=None):
    base_url = "https://api.themoviedb.org/3"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Accept": "application/json"
    }
    params = params or {}
    response = requests.get(f"{base_url}/{endpoint}", params=params, headers=headers)
    return response.json()

In [6]:
def add_person(movie_uri,person_member,graph):
        person_id = f"Person_{person_member['id']}"
        person_uri = URIRef(MOVIE_NS[person_id])
        g.add((person_uri, RDF.type, MOVIE.Person))
        if (person_uri, None, None) in graph:
            existing_name = next(graph.objects(person_uri, MOVIE_NS.name), None)
            if existing_name:
                new_name = person_member['name']
                if new_name != existing_name:
                    graph.set((person_uri, MOVIE_NS.name, Literal(new_name, datatype=XSD.string)))
            else:
                graph.add((person_uri, MOVIE_NS.name, Literal(person_member['name'], datatype=XSD.string)))

            if 'nationality' in person_member:
                existing_nationality = next(graph.objects(person_uri, MOVIE_NS.nationality), None)
                if existing_nationality:
                    new_nationality = person_member['nationality']
                    if new_nationality != existing_nationality:
                        graph.set((person_uri, MOVIE_NS.nationality, Literal(new_nationality, datatype=XSD.string)))
                else:
                    graph.add((person_uri, MOVIE_NS.nationality, Literal(person_member['nationality'], datatype=XSD.string)))

            if 'birthday' in person_member:
                existing_birth_date = next(graph.objects(person_uri, MOVIE_NS.birthDate), None)
                if existing_birth_date:
                    new_birth_date = person_member['birthday']
                    if new_birth_date != existing_birth_date:
                        graph.set((person_uri, MOVIE_NS.birthDate, Literal(new_birth_date, datatype=XSD.date)))
                else:
                    graph.add((person_uri, MOVIE_NS.birthDate, Literal(person_member['birthday'], datatype=XSD.date)))

            if 'place_of_birth' in person_member:
                existing_place_of_birth = next(graph.objects(person_uri, MOVIE_NS.placeOfBirth), None)
                if existing_place_of_birth:
                    new_place_of_birth = person_member['place_of_birth']
                    if new_place_of_birth != existing_place_of_birth:
                        graph.set((person_uri, MOVIE_NS.placeOfBirth, Literal(new_place_of_birth, datatype=XSD.string)))
                else:
                    graph.add((person_uri, MOVIE_NS.placeOfBirth, Literal(person_member['place_of_birth'], datatype=XSD.string)))

            if 'gender' in person_member:
                existing_gender = next(graph.objects(person_uri, MOVIE_NS.gender), None)
                if existing_gender:
                    new_gender = person_member['gender']
                    if new_gender != existing_gender:
                        graph.set((person_uri, MOVIE_NS.gender, Literal(new_gender, datatype=XSD.string)))
                else:
                    graph.add((person_uri, MOVIE_NS.gender, Literal(person_member['gender'], datatype=XSD.string)))

In [7]:
def add_persons_to_graph(movie_uri, person_data, job, graph):
    for person_member in person_data:
        if job == "Actor":
            person_id = f"Person_{person_member['id']}"
            person_uri = URIRef(MOVIE_NS[person_id])
            g.add((person_uri, RDF.type, MOVIE.Person))
            g.add((movie_uri, MOVIE_NS.hasActor, person_uri))
            g.add((person_uri, MOVIE_NS.actedIn, movie_uri))
            add_person(movie_uri,person_member,graph)
        elif job == "Director of Photography":
            person_id = f"Person_{person_member['id']}"
            person_uri = URIRef(MOVIE_NS[person_id])
            g.add((person_uri, RDF.type, MOVIE.Person))
            g.add((movie_uri, MOVIE_NS.filmedBy, person_uri))
            g.add((person_uri, MOVIE_NS.cinematographyFor, movie_uri))
            add_person(movie_uri,person_member,graph)
        elif job == "Producer":
            person_id = f"Person_{person_member['id']}"
            person_uri = URIRef(MOVIE_NS[person_id])
            g.add((person_uri, RDF.type, MOVIE.Person))
            g.add((movie_uri, MOVIE_NS.filmedBy, person_uri))
            #print(person_uri)
            g.add((person_uri, MOVIE_NS.cinematographyFor, movie_uri))
            add_person(movie_uri,person_member,graph)
        elif job == "Director":
            person_id = f"Person_{person_member['id']}"
            person_uri = URIRef(MOVIE_NS[person_id])
            g.add((person_uri, RDF.type, MOVIE.Person))
            g.add((movie_uri, MOVIE_NS.directedBy, person_uri))
            g.add((person_uri, MOVIE_NS.directs, movie_uri))
            add_person(movie_uri,person_member,graph)
        elif job == "Original Music Composer":
            person_id = f"Person_{person_member['id']}"
            person_uri = URIRef(MOVIE_NS[person_id])
            g.add((person_uri, RDF.type, MOVIE.Person))
            g.add((movie_uri, MOVIE_NS.hasSoundtrackBy, person_uri))
            g.add((person_uri, MOVIE_NS.composedMusicFor, movie_uri))
            add_person(movie_uri,person_member,graph)
        elif job == "Screenplay":
            person_id = f"Person_{person_member['id']}"
            person_uri = URIRef(MOVIE_NS[person_id])
            g.add((person_uri, RDF.type, MOVIE.Person))
            g.add((movie_uri, MOVIE_NS.writtenBy, person_uri))
            g.add((person_uri, MOVIE_NS.wroteScriptFor, movie_uri))
            add_person(movie_uri,person_member,graph)

In [8]:
from rdflib import Graph, Namespace, URIRef, RDF, Literal, XSD

def add_movie_genre_association(movie_uri, movie_data, graph):
    if 'genres' in movie_data:
        for genre in movie_data['genres']:
            genre_name = genre['name'].replace(" ", "_")
            genre_uri = URIRef(MOVIE_NS[genre_name])
         #   print(genre_name)
         #   print(genre_uri)
            if (genre_uri, None, None) not in graph:
                graph.add((genre_uri, RDF.type, MOVIE_NS.Genre))
                graph.add((genre_uri, MOVIE_NS.genreName, Literal(genre['name'], datatype=XSD.string)))

        for genre in movie_data['genres']:
            genre_name = genre['name'].replace(" ", "_")
            genre_uri = URIRef(MOVIE_NS[genre_name])
            graph.add((movie_uri, MOVIE_NS.genreOf, genre_uri))

    return graph


In [9]:
def add_characters_to_graph(movie_uri,movie_id, credits_data, graph):
    if 'cast' in credits_data:
        for character_info in credits_data['cast']:
            character_name = character_info.get('character', '')
            actor_id = character_info.get('id', None)

            if not character_name or actor_id is None:
                continue

            character_id = f"Character_{actor_id}_{movie_id}"
            character_uri = URIRef(MOVIE_NS[character_id])

            actor_id = f"Person_{actor_id}"
            actor_uri = URIRef(MOVIE_NS[actor_id])

            graph.add((character_uri, RDF.type, MOVIE_NS.Character))
            graph.add((character_uri, MOVIE_NS.characterName, Literal(character_name, datatype=XSD.string)))
            graph.add((movie_uri, MOVIE_NS.hasCharacter, character_uri))
            graph.add((character_uri, MOVIE_NS.appearsIn, movie_uri))

            graph.add((character_uri, MOVIE_NS.portrayedBy, actor_uri))
            graph.add((actor_uri, MOVIE_NS.portrays, character_uri))
    return graph

In [10]:
def fetch_and_add_movie_awards_to_graph(movie_title, movie_uri, graph):
    def get_wikidata_id_by_title(movie_title):
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        query = """
        SELECT ?movie WHERE {
          ?movie wdt:P31 wd:Q11424;
                 rdfs:label "%s"@en.
        } LIMIT 1
        """ % movie_title.replace('"', '\\"')

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            return result["movie"]["value"].split('/')[-1]
        return None

    def get_movie_awards(wikidata_id):
        """Fetches the awards for a movie based on its Wikidata ID."""
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        query = """
        SELECT ?awardLabel WHERE {
          wd:%s p:P166 ?awardStatement .
          ?awardStatement ps:P166 ?award .
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        """ % wikidata_id

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        awards = [result['awardLabel']['value'] for result in results["results"]["bindings"]]
        return awards
    wikidata_id = get_wikidata_id_by_title(movie_title)
    if not wikidata_id:
        print(f"No Wikidata entry found for {movie_title}")
        return

    awards = get_movie_awards(wikidata_id)

    for award in awards:
        award_uri = URIRef(f"http://www.wikidata.org/entity/{award.replace(' ', '_')}")
        graph.add((award_uri, RDF.type, MOVIE.Award))
        graph.add((award_uri, MOVIE.awardedTo, movie_uri))
        graph.add((movie_uri, MOVIE.hasAward, award_uri))
        graph.add((award_uri, RDFS.label, Literal(award)))
        graph.add((award_uri, MOVIE.awardName, Literal(award)))

In [11]:
def add_filmedAt_locations_to_graph(movie_uri, movie_data, graph):

    if 'production_countries' in movie_data:
        countries_data = movie_data['production_countries']
        print(movie_data['production_countries'])
        for country_info in countries_data:
            country_name = country_info.get('name', '')

            if not country_name:
                continue

            location_id = f"Location_{country_name}_{movie_uri}"
            location_uri = URIRef(MOVIE_NS[location_id])

            country_id = f"Country_{country_name}"
            country_uri = URIRef(MOVIE_NS[country_id])

            graph.add((location_uri, RDF.type, MOVIE_NS.Location))
            graph.add((location_uri, MOVIE_NS.name, Literal(country_name, datatype=XSD.string)))
            graph.add((movie_uri, MOVIE_NS.filmedAt, location_uri))

            graph.add((location_uri, MOVIE_NS.isInCountry, country_uri))

    return graph

In [12]:
def create_movie_rdf_graph(movie_id):
    MOVIE = Namespace("http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#")
    movie_data = tmdb_api_request(f"movie/{movie_id}")
    movie_id_in = f"Movie_{movie_id}"
    movie_uri = URIRef(MOVIE[str(movie_id_in)])
    if 'success' in movie_data:
        return
    else:
        pass
    if (movie_uri, None, None) in g:
        print(f"Movie {movie_id} already exists in the graph.")
    else:
        g.add((movie_uri, RDF.type, MOVIE.Movie))
        g.add((movie_uri, MOVIE.title, Literal(movie_data['title'], datatype=XSD.string)))
        g.add((movie_uri, MOVIE.releaseDate, Literal(movie_data['release_date'], datatype=XSD.date)))
        g.add((movie_uri, MOVIE.budget, Literal(movie_data['budget'], datatype=XSD.int)))
        g.add((movie_uri, MOVIE.originalLanguage, Literal(movie_data['original_language'], datatype=XSD.string)))
    credits_data = tmdb_api_request(f"movie/{movie_id}/credits")

    fetch_and_add_movie_awards_to_graph(movie_data['title'], movie_uri, g)
    #add_filmedAt_locations_to_graph(movie_uri,movie_data,g)
    add_characters_to_graph(movie_uri, movie_id, credits_data, g)

    for cast_member in credits_data['cast']:
        add_persons_to_graph(movie_uri, [cast_member], "Actor", g)

    for crew_member in credits_data['crew']:
        job = crew_member['job']
        add_persons_to_graph(movie_uri, [crew_member], job, g)
    vote_average = movie_data.get('vote_average', None)

    if vote_average is not None:
        rating_property = MOVIE_NS.rating
        g.add((movie_uri, rating_property, Literal(vote_average, datatype=XSD.float)))
    return g

In [23]:
movie_id = '207'


create_movie_rdf_graph(movie_id)

create_movie_rdf_graph(movie_id)


Movie 207 already exists in the graph.


KeyboardInterrupt: ignored

In [13]:
from rdflib import Graph, RDF, RDFS, OWL

In [14]:
import concurrent.futures
from tqdm import tqdm

num_workers = 20

ids = range(1, 2001)
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = [executor.submit(create_movie_rdf_graph, movie_id) for movie_id in ids]

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(ids), desc="Processing Movies"):
        try:
            result = future.result()
        except Exception as e:
            print(f"Error processing movie: {e}")

Processing Movies:   0%|          | 5/2000 [00:06<37:34,  1.13s/it]

No Wikidata entry found for Four Rooms
No Wikidata entry found for Life in Loops (A Megacities RMX)


Processing Movies:   0%|          | 8/2000 [00:07<17:50,  1.86it/s]

No Wikidata entry found for Finding Nemo


Processing Movies:   1%|          | 20/2000 [00:07<03:51,  8.55it/s]

...

No Wikidata entry found for Golden Door


Processing Movies:  99%|█████████▉| 1988/2000 [08:08<00:02,  4.88it/s]

No Wikidata entry found for Lara Croft: Tomb Raider - The Cradle of Life


Processing Movies: 100%|█████████▉| 1997/2000 [09:31<00:00,  3.50it/s]


KeyboardInterrupt: ignored

In [15]:
from rdflib.namespace import RDF, VOID, XSD

In [19]:
!pip install rdflib-void

ERROR: Could not find a version that satisfies the requirement rdflib-void (from versions: none)
ERROR: No matching distribution found for rdflib-void


In [28]:
import rdflib.void as vo

In [46]:
from rdflib import Graph, Namespace
from rdflib.plugins.sparql import prepareQuery

In [53]:
MOVIE = Namespace("http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#")

movie_count = len(set(g.subjects(RDF.type, MOVIE.Movie)))

person_count = len(set(g.subjects(RDF.type, MOVIE.Person)))

char_count = len(set(g.subjects(RDF.type,MOVIE.CHARACTER)))

hasAward_count = len(set(g.triples((None, MOVIE.hasAward, None))))

hasActor_count = len(set(g.triples((None, MOVIE.hasActor, None))))

movie_count, person_count, char_count, hasAward_count, hasActor_count

(1354, 42476, 0, 1527, 52279)

In [37]:
a , b = vo.generateVoID(g)

In [38]:
a

<Graph identifier=N1da040b761544697ae568655741b12ff (<class 'rdflib.graph.Graph'>)>

In [43]:
b

rdflib.term.URIRef('http://example.org/Dataset')

In [42]:
a.serialize(destination="sm.rdf", format='application/rdf+xml')

<Graph identifier=N1da040b761544697ae568655741b12ff (<class 'rdflib.graph.Graph'>)>

In [54]:
query = """
PREFIX : <http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#>

SELECT ?actor (COUNT(?movie) AS ?movieCount)
WHERE {
    ?movie :hasActor ?actor .
}
GROUP BY ?actor
HAVING (COUNT(?movie) > 5)
"""

for row in g.query(query):
    print(f"Actor: {row.actor}, Movie Count: {row.movieCount}")

Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_3129, Movie Count: 6
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_3131, Movie Count: 8
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_3136, Movie Count: 10
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_138, Movie Count: 7
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_62, Movie Count: 16
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_1979, Movie Count: 7
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_53, Movie Count: 9
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_1642, Movie Count: 6
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_1646, Movie Count: 7
Actor: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_1640

In [26]:
query = """
PREFIX : <http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#>

SELECT ?movieTitle (GROUP_CONCAT(DISTINCT ?role;separator=", ") AS ?roles) ?personName
WHERE {
    {
        ?movie :hasActor ?person ;
               :title ?movieTitle .
        ?person :name ?personName .
        BIND("Actor" AS ?role)
    } UNION {
        ?movie :filmedBy ?person ;
               :title ?movieTitle .
        ?person :name ?personName .
        BIND("Director" AS ?role)
    } UNION {
        ?movie :producedBy ?person ;
               :title ?movieTitle .
        ?person :name ?personName .
        BIND("Producer" AS ?role)
    } UNION {
        ?movie :writtenBy ?person ;
               :title ?movieTitle .
        ?person :name ?personName .
        BIND("Writer" AS ?role)
    }
}
GROUP BY ?movieTitle ?personName
HAVING (COUNT(DISTINCT ?role) > 1)
"""
for row in g.query(query):
    print(f"Movie: {row.movieTitle}, Person: {row.personName}, Roles: {row.roles}")


Movie: Frida, Person: Salma Hayek Pinault, Roles: Actor, Director
Movie: Jackie Brown, Person: Quentin Tarantino, Roles: Actor, Writer
Movie: Pulp Fiction, Person: Quentin Tarantino, Roles: Actor, Writer
Movie: From Dusk Till Dawn, Person: Quentin Tarantino, Roles: Actor, Writer
Movie: Death Proof, Person: Quentin Tarantino, Roles: Actor, Director, Writer
Movie: Planet Terror, Person: Quentin Tarantino, Roles: Actor, Director
Movie: Four Rooms, Person: Lawrence Bender, Roles: Actor, Director
Movie: Reservoir Dogs, Person: Lawrence Bender, Roles: Actor, Director
Movie: Pulp Fiction, Person: Lawrence Bender, Roles: Actor, Director
Movie: American Beauty, Person: Bruce Cohen, Roles: Actor, Director
Movie: Cars, Person: Joe Ranft, Roles: Actor, Writer
Movie: Finding Nemo, Person: Andrew Stanton, Roles: Actor, Writer
Movie: Toy Story 2, Person: Andrew Stanton, Roles: Actor, Writer
Movie: Toy Story, Person: Andrew Stanton, Roles: Actor, Writer
Movie: Finding Nemo, Person: Bob Peterson, Roles

In [55]:
query = """
PREFIX : <http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?director ?directorName (COUNT(?movie) AS ?movieCount)
WHERE {
    ?movie :directedBy ?director ;
           :rating ?rating .
    ?director :name ?directorName .
    FILTER (xsd:float(?rating) > 8)
}
GROUP BY ?director ?directorName
HAVING (COUNT(?movie) > 0)
"""

for row in g.query(query):
    print(f"Director: {row.director}, Name: {row.directorName}, Number of Movies with Vote Average in TMDB > 8: {row.movieCount}")

Director: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_138, Name: Quentin Tarantino, Number of Movies with Vote Average in TMDB > 8: 2
Director: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_39, Name: Sam Mendes, Number of Movies with Vote Average in TMDB > 8: 1
Director: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_68, Name: Fritz Lang, Number of Movies with Vote Average in TMDB > 8: 2
Director: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_1, Name: George Lucas, Number of Movies with Vote Average in TMDB > 8: 1
Director: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_59, Name: Luc Besson, Number of Movies with Vote Average in TMDB > 8: 1
Director: http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#Person_24, Name: Robert Zemeckis, Number of Movies with Vote Average in TMDB > 8: 2
Director: http://www.semanticweb.org/jalal/ontolog

##Main parse function

In [ ]:
sparql_query = """
PREFIX : <http://www.semanticweb.org/jalal/ontologies/2023/10/movie-ontology#>

SELECT ?movie1 ?movie1Name ?movie2 ?movie2Name ?movie3 ?movie3Name (COUNT(DISTINCT ?actor) AS ?sharedActors)
WHERE {
    ?movie1 :hasActor ?actor ; :title ?movie1Name .
    ?movie2 :hasActor ?actor ; :title ?movie2Name .
    ?movie3 :hasActor ?actor ; :title ?movie3Name .
    FILTER (?movie1 != ?movie2 && ?movie2 != ?movie3 && ?movie1 != ?movie3)
}
GROUP BY ?movie1 ?movie1Name ?movie2 ?movie2Name ?movie3 ?movie3Name
ORDER BY DESC(?sharedActors)
LIMIT 10
"""


for row in g.query(sparql_query):
    print(f"Movie 1: {row.movie1Name}, Movie 2: {row.movie2Name}, Movie 3: {row.movie3Name}, Shared Actors: {row.sharedActors}")

In [ ]:
!pip install owlrl

In [ ]:
import owlrl

In [ ]:
from rdflib import  OWL

In [ ]:
from itertools import combinations

In [ ]:
workedWith = MOVIE.workedWith
g.add((workedWith, RDF.type, OWL.SymmetricProperty))

In [ ]:
def add_worked_with():
  for movie in g.subjects(RDF.type, MOVIE.Movie):
      individuals = set()

      for actor in g.objects(movie, MOVIE.hasActor):
          individuals.add(actor)
      for director in g.objects(movie, MOVIE.directedBy):
          individuals.add(director)
      for writer in g.objects(movie, MOVIE.writtenBy):
          individuals.add(writer)

      for individual1, individual2 in combinations(individuals, 2):
          g.add((individual1, workedWith, individual2))
          if True:
            return

In [ ]:
add_worked_with()
for subj, pred, obj in g.triples((None, workedWith, None)):
    print(f"{subj} worked with {obj}")
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics).expand(g)
for subj, pred, obj in g.triples((None, workedWith, None)):
    print(f"{subj} worked with {obj}")